# Analysing `contacts.csv`

### are there any differences between all `contacts.csv` dup records?

In [2]:
import csv
c_id__row = {}
id_diffs  = {}
id_dups   = {}
key       = "primary_contact.id"
with open('contacts.csv') as csvfile:
    reader = csv.DictReader(csvfile,delimiter=";")
    for row in reader:
        _row_key = row[key]
        _row = c_id__row.get(_row_key,{})
        if row!=_row:
            _=id_diffs.setdefault(_row_key,0)
            id_diffs[_row_key]+=1
            c_id__row[_row_key]=row
        else:
            _=id_dups.setdefault(_row_key,0)
            id_dups[_row_key]+=1

#unique data for each id
id_diffs = {
    _id:_diffs
    for _id,_diffs in id_diffs.items()
    if _diffs>1
}            

if not id_diffs:
    print("data is equal across all dups, no updates in any row")

data is equal across all dups, no updates in any row


### Are there any contact info missing in `places.csv`

In [1]:

import csv

files = [
    'places.csv','contacts.csv'
]
key       = "primary_contact.id"
file__ids = {}
for f in files:
    file__ids[f]=set()
    with open(f, encoding='latin-1') as csvfile:
        reader = csv.DictReader(csvfile,delimiter=";")
        for row in reader:
            row_key = row[key]
            #skip empty values
            if row_key.strip():
                file__ids[f].add(row_key)


places_ids   = file__ids['places.csv']
contacts_ids = file__ids['contacts.csv']
print ("common primary_contact ids : {}".format(
        len(contacts_ids.intersection(places_ids))
    )
)
print ("extra primary_contact from contacts: {}".format(
        len(contacts_ids-places_ids)
    )
)
print ("missing primary_contact from places: {}".format(
        len(places_ids-contacts_ids)
    )
)

common primary_contact ids : 163
extra primary_contact from contacts: 467
missing primary_contact from places: 0


### First conclusion: `contacts.csv` is a *redundant* data file

# Data shapping

Let's prepare `places.csv` for elastic-kibana insertion. We're gonna create a well-formated data file since in csv numbers and booleans are stored as text as long as comma-separated values are stored as a chunk of text.     
In order to allow a good drill-down filtering we must split comma-separated joined-value in lists of values.     

#### Enrichment
- geopoints should be joined by comma in a single field, instead of having 2 separated values.
- In order to build an state-map of inputs, let's standarize the `input_state` field to `input_state_fips` (https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code)

In [8]:

import csv
import ndjson
import json
_file   = "places.csv"
_output = "data_ingest.ndjson"


def guess_type(s):
    try:
        value = json.loads(s)
    except ValueError:
        return str
    else:
        return type(value)

fips_mapping = {
    "california"   : "CA",
    "texas"        : "TX",
    "florida"      : "FL",
    "colorado"     : "CO",
    "washington"   : "WA",
    "ohio"         : "OH",
    "pennsylvania" : "PA",
    "new_york"     : "NY",
    "NULL"         : None,
    "nevada"       : "NV",
    "arizona"      : "AZ",
}
geo_points = ["geocoordinate","manual_geocoordinate"]
split_values = [
    "matched_rules",#['address', ' phone']
    "sic_code_ids",#['336501', '354405']
    "labels.sic_code_ids",#['Foundries-Aluminum Brass Bronze (Mfrs)', 'Molds (Mfrs)']
    "naics_code_ids",#['33152304', '33351410']
    "labels.naics_code_ids",#['Foundries-Aluminum Brass Bronze (Mfrs)', 'Molds (Mfrs)']
    "payment_types",#['Visa', 'MasterCard', 'American Express', 'Discover', 'Check', 'Cash']
    "labels.payment_types",#['Visa', 'Mastercard', 'American Express', 'Discover', 'Check', 'Cash']
    "record_list_ids",#['3054', '4902']
    "website_keywords",#['brushholder castings', 'brushholder rebuild', 'brushholder repair/rebuild', 'brushholders', 'carbon brush', 'carbon brush for commutator', 'commutator brushes', 'constant force springs', 'diamond brushholders', 'diamond carbon', 'electric carbon graphite', 'flat wire springs', 'fulmer company motor brushes', 'leveling wedges', 'locomotive', 'mill motor and mine haulage', 'pantographs', 'perma cast', 'plating lines', 'pressure finger assemblies', 'round wire springs', 'shunts', 'slip ring brushes', 'slip rings', 'specialty machining castings carbon', 'spring assemblies', 'transit']
    "historical_names",#['Memorial Hermann Surgical Hosp', 'Memorial Hermann Surgical Hospital']
    "business_type_ids",#['329ff482-23a0-11e2-94ad-a9a9b1ace525', '32a263de-23a0-11e2-9ddf-ffc48ffb142a', '32a6ec2e-23a0-11e2-9745-41f2c6dbe07f']
    "labels.business_type_ids",#['Emergency Room', 'For Profit', 'General Medical/Surgical']
    "professional_specialty_ids",#['31fc0700-23a0-11e2-820f-18f582d88ad5', '303d4fa0-23a0-11e2-871e-dfd32e4bbfd5', '30640dd4-23a0-11e2-8f70-ca85a13a8964']
    "labels.professional_specialty_ids",#['Family Practice', 'Internal Medicine', 'Pulmonary Disease']
    "operating_hours_description",#['Tue 10am-3pm', ' Fri-Sat 10am-3pm', ' Wed-Thu 1pm-7pm']
    "input_email",#['sunithab.09@gmail.com', ' sunitha@momgeni.com']
    "input_name",#['Alex Samples (exec assistant)', ' Lauren Adams (Boss) ']
    "eins",#['204496500', '943130371']
    "suppressed_fields",#['street', 'building_number', 'carrier_route_code', 'census_block', 'census_tract', 'delivery_point_bar_code', 'entry_point_match_level_restricted', 'geo_match_level', 'infogroup_match_type_code', 'lat', 'latitude', 'latitude_entry_point_restricted', 'lon', 'longitude', 'longitude_entry_point_restricted', 'suite', 'zip_four', 'mailing_score_code', 'mailing_confidence_code', 'manual_geocoordinate', 'landmark_address']
    "insurances_accepted",#['aetna', 'dental dppo']
    "primary_contact.title_codes",#['C', '1', '2']
    "labels.primary_contact.title_codes",#['Chief Executive Officer', 'Owner', 'President']
    "ancestor_headquarters_ids",#['408126506', '002532471']
    "languages_spoken",#['english', 'spanish']
    "labels.languages_spoken",#['English', 'Spanish']
]

with open(_file, encoding='latin-1') as csvfile,open(_output, 'w') as f:
    reader = csv.DictReader(csvfile,delimiter=";")
    writer = ndjson.writer(f, ensure_ascii=False)
    for row in reader:
        item = {}
        for k,v in row.items():
            if k in split_values:
                v = [_v.strip() for _v in v.split(",")]
            else:
                _type = guess_type(v)
                v = _type(v)
            if v:
                item[k]=v

        #enrichment data
        #geo_points
        for k in geo_points:
            _lat,_lon = item.pop('{}.lat'.format(k),'') , item.pop('{}.lon'.format(k),'')
            if _lat and _lon:
                item[k]="{},{}".format(_lat , _lon)

        #standarize input states
        _input_state = item.get('input_state','')
        if _input_state and fips_mapping.get(_input_state,None):
            item['input_state_fips'] = fips_mapping.get(_input_state,None)
        
        writer.writerow(item)



## Kibana
Kibana helps you import data, however this time it has to be tunned replacing all `text` by `keywords` mappings unless `company_description` field that is free text. `keyword` typed fields allows aggregations, the key feature in this POC.    
Also pay attention to location fields `geocoordinate` and `manual_geocoordinate`, they should be mapped as `geo_point` in order to build maps using them.
The wizard-created index pattern should be replaced to non-time-basis one